In [2]:
from mistralai import Mistral
from mistralai.models import UserMessage
from dotenv import load_dotenv
import os
import json

# Preparation

### Generate fake weather data

In [3]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

In [4]:
get_weather('Cairo')

7.2

# Q1. Define function description

In [5]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get the temperature for a given city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name the city to get its temperature."
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [6]:
get_weather_tool = {
    "type": "function",
    "function": {           
        "name": "get_weather",
        "description": "Get the temperature for a given city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "Name the city to get its temperature."
                }
            },
            "required": ["city"],
            "additionalProperties": False
        }
    }
}


In [7]:
load_dotenv()

True

In [8]:
api_key = os.getenv("API_KEY")

In [9]:
client = Mistral(api_key = api_key )

In [10]:
question = "What's the current temperature in Berlin, Germany?"

system_prompt = """
You are a weather program. 
Your task is to fetch the current temperature for the specified city in Celsius.
The city will be provided by the user. Make sure to only return the temperature.
""".strip()

tools = [get_weather_tool]

chat_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question}
]

response = client.chat.complete(
    model = "mistral-medium-latest",
    messages = chat_messages, 
    tools = tools,
    tool_choice = "any", 
    parallel_tool_calls = False,
)
response.choices



[ChatCompletionChoice(index=0, message=AssistantMessage(content='', tool_calls=[ToolCall(function=FunctionCall(name='get_weather', arguments='{"city": "Berlin"}'), id='LaPWlw939', type=None, index=0)], prefix=False, role='assistant'), finish_reason='tool_calls')]

In [11]:
calls = response.choices
call = calls[0]
f_name = call.message.tool_calls[0].function.name
arguments = json.loads(call.message.tool_calls[0].function.arguments)

In [12]:
f = globals()[f_name]

In [13]:
search_results = f(**arguments)
search_results

20.0

# Q2. Adding another tool

In [14]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [15]:
# Add description 

set_weather_tool = {
    "type": "function",
    "function": {
        "name": "set_weather",
        "description": "Add or update the temperature (°C) for a specified city in the weather database.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "Name of the city to add or update in the weather database"
                },
                "temp": {
                    "type": "number",
                    "description": "Temperature in Celsius to set for the city"
                }
            },
        "required": ["city", "temp"],
        "additionalProperties": False
        }
    }
}



In [16]:
question = "What's the current temperature in berlin?"

system_prompt = """
You are a data assistant. 
Your task is to fetch the current temperature from get_weather_tool.
save the result in the database.
""".strip()

tools = [get_weather_tool, set_weather_tool]

chat_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question}
]

response = client.chat.complete(
    model = "mistral-medium-latest",
    messages = chat_messages, 
    tools = tools,
    tool_choice = "any", 
    parallel_tool_calls = False,
)
response.choices



[ChatCompletionChoice(index=0, message=AssistantMessage(content='', tool_calls=[ToolCall(function=FunctionCall(name='get_weather', arguments='{"city": "Berlin"}'), id='T61QPnS8G', type=None, index=0)], prefix=False, role='assistant'), finish_reason='tool_calls')]

# Q3. Install FastMCP

In [17]:
pip install fastmcp --trusted-host pypi.org --trusted-host files.pythonhosted.org

Note: you may need to restart the kernel to use updated packages.


In [18]:
!fastmcp --version 

2.10.5


# Q4. Simple MCP Server

The code have been saved in weather_server.py and run it by (fastmcp run weather_server.py)

and the output is 


[07/15/25 13:20:11] INFO     Starting MCP server 'Demo 🚀' with transport 'stdio

# Q5. Protocol

{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city": "Berlin"}}}


OUTPUT:


{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}

# Q6. Client

In [19]:
from fastmcp import Client
import weather_server

In [20]:
async def main():
    async with Client("weather_server.py") as mcp_client:
        tools = await mcp_client.list_tools()
        print("Available tools:")
        for tool in tools:
            print(tool)

await main()

Available tools:
name='get_weather' title=None description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.    ' inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'} outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True} annotations=None meta=None
name='set_weather' title=None description="Sets the temperature for a specified city.\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update." inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Temp'